In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm

train_data = pd.read_csv('features.csv',index_col='match_id')
test_data = pd.read_csv('features_test.csv',index_col='match_id')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
columns_with_nan = train_data.columns[train_data.isna().any()].tolist()
columns_with_nan, len(columns_with_nan)

(['first_blood_time',
  'first_blood_team',
  'first_blood_player1',
  'first_blood_player2',
  'radiant_bottle_time',
  'radiant_courier_time',
  'radiant_flying_courier_time',
  'radiant_first_ward_time',
  'dire_bottle_time',
  'dire_courier_time',
  'dire_flying_courier_time',
  'dire_first_ward_time'],
 12)

In [8]:
for column in columns_with_nan:
    train_data[column].fillna(0, inplace=True)
    test_data[column].fillna(0, inplace=True)
    
x_train = np.array(train_data.values[:,1:-6])
y_train = np.array(train_data.values[:,-5])

In [5]:
from collections import Counter
sum(x for x in Counter(train_data.count()).values() if x != 96)

12

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score

num_trees = [5,10,20,30,40,50]
results = {}

splitter = KFold(n_splits=5, random_state=241, shuffle=True)
for num in num_trees:
    clf = GradientBoostingClassifier(n_estimators=num,learning_rate=0.1,verbose=False,random_state=241)
    # clf.fit(x_train, y_train)
    result = cross_val_score(estimator=clf, X=x_train, y=y_train, scoring='roc_auc' ,cv=splitter,n_jobs=-1)
    results[num] =  result
results

{5: array([0.64010969, 0.62849414, 0.63656996, 0.63369292, 0.63982151]),
 10: array([0.66943496, 0.65627754, 0.66390454, 0.66281223, 0.66950933]),
 20: array([0.68939011, 0.67567277, 0.68201026, 0.67977584, 0.68741889]),
 30: array([0.69641668, 0.68365441, 0.68731862, 0.68728458, 0.69280674]),
 40: array([0.70087493, 0.68889686, 0.69145856, 0.69219074, 0.69727685]),
 50: array([0.70374327, 0.69328481, 0.69471307, 0.69565719, 0.7000093 ])}

In [13]:
import time
import datetime

start_time = datetime.datetime.now()

clf = GradientBoostingClassifier(n_estimators=30,learning_rate=0.1,verbose=False,random_state=241)
result = cross_val_score(estimator=clf, X=x_train, y=y_train, scoring='roc_auc' ,cv=splitter,n_jobs=-1)

print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:01:57.183913


In [17]:
sum(x for x in results[50]) / 5. 

0.6974815295110693

Второй этап выполнения проекта

In [21]:
from sklearn.linear_model import LogisticRegression
__C__ = np.power(10.0, np.arange(-5, 6))
results = {}
for c in tqdm(__C__):
    tic = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C=c, n_jobs=-1)
    results[c] = cross_val_score(estimator=clf, X=x_train, y=y_train, scoring='roc_auc', cv=splitter, n_jobs=-1)
    toc = datetime.datetime.now()
    print(c,'  ',toc-tic)
results


  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

  9%|▉         | 1/11 [00:49<08:14, 49.44s/it]

1e-05    0:00:49.435226


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 18%|█▊        | 2/11 [02:04<09:18, 62.00s/it]

0.0001    0:01:14.569159


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 27%|██▋       | 3/11 [03:43<09:57, 74.63s/it]

...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 91%|█████████ | 10/11 [15:40<01:34, 94.06s/it]

10000.0    0:01:46.997104


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

100%|██████████| 11/11 [17:45<00:00, 96.82s/it]


100000.0    0:02:04.421134


{1e-05: array([0.72065812, 0.71285547, 0.71457292, 0.71445084, 0.71707339]),
 0.0001: array([0.72138949, 0.71372825, 0.71482316, 0.71515479, 0.71738799]),
 0.001: array([0.7213844 , 0.71362729, 0.71466599, 0.71518145, 0.71724775]),
 0.01: array([0.72147452, 0.71373747, 0.71452501, 0.71518744, 0.71727707]),
 0.1: array([0.72137792, 0.71361573, 0.71458591, 0.71518446, 0.7171182 ]),
 1.0: array([0.7214501 , 0.71363065, 0.7144626 , 0.71517563, 0.71704145]),
 10.0: array([0.7213331 , 0.71378024, 0.71444145, 0.71519719, 0.71716057]),
 100.0: array([0.72137342, 0.71352513, 0.71458744, 0.71515298, 0.71724849]),
 1000.0: array([0.72139818, 0.71359729, 0.71445409, 0.71511816, 0.71706459]),
 10000.0: array([0.72139351, 0.71358458, 0.71466798, 0.71514257, 0.71720643]),
 100000.0: array([0.72134835, 0.71354494, 0.7144344 , 0.71516977, 0.71704921])}

In [22]:
best_c = 1e-5
best_score = 0
for c in results:
    score = max(results[c])
    if score > best_score:
        best_c, best_score = c, score
best_c, best_score

(0.01, 0.7214745154308478)

In [29]:
reduce_data = train_data.drop(columns=['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                                                    'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])
x_train = np.array(train_data.values[:,1:-6])
y_train = np.array(train_data.values[:,-5])

In [30]:
results = {}
for c in tqdm(__C__):
    tic = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C=c, n_jobs=-1)
    results[c] = cross_val_score(estimator=clf, X=x_train, y=y_train, scoring='roc_auc', cv=splitter, n_jobs=-1)
    toc = datetime.datetime.now()
    print(c,'  ',toc-tic)
results


  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

  9%|▉         | 1/11 [00:50<08:20, 50.02s/it]

1e-05    0:00:50.014775


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 18%|█▊        | 2/11 [02:04<09:19, 62.16s/it]

0.0001    0:01:14.305160


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 27%|██▋       | 3/11 [03:43<09:56, 74.61s/it]

...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 91%|█████████ | 10/11 [15:26<01:32, 92.68s/it]

10000.0    0:01:36.531458


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

100%|██████████| 11/11 [17:20<00:00, 94.58s/it]


100000.0    0:01:53.636976


{1e-05: array([0.72065812, 0.71285547, 0.71457292, 0.71445084, 0.71707339]),
 0.0001: array([0.72138949, 0.71372825, 0.71482316, 0.71515479, 0.71738799]),
 0.001: array([0.7213844 , 0.71362729, 0.71466599, 0.71518145, 0.71724775]),
 0.01: array([0.72147452, 0.71373747, 0.71452501, 0.71518744, 0.71727707]),
 0.1: array([0.72137792, 0.71361573, 0.71458591, 0.71518446, 0.7171182 ]),
 1.0: array([0.7214501 , 0.71363065, 0.7144626 , 0.71517563, 0.71704145]),
 10.0: array([0.7213331 , 0.71378024, 0.71444145, 0.71519719, 0.71716057]),
 100.0: array([0.72137342, 0.71352513, 0.71458744, 0.71515298, 0.71724849]),
 1000.0: array([0.72139818, 0.71359729, 0.71445409, 0.71511816, 0.71706459]),
 10000.0: array([0.72139351, 0.71358458, 0.71466798, 0.71514257, 0.71720643]),
 100000.0: array([0.72134835, 0.71354494, 0.7144344 , 0.71516977, 0.71704921])}

In [31]:
best_c = 1e-5
best_score = 0
for c in results:
    score = max(results[c])
    if score > best_score:
        best_c, best_score = c, score
best_c, best_score

(0.01, 0.7214745154308478)